# Project Overview
Flux balance analysis (FBA) is a mathematical approach for simulating metabolism in genome-scale reconstructions of metabolic networks. These network reconstructions contain all of the known metabolic reactions in an organism and the genes that encode various enzymes. Using these networks, the following predictions can be made:  
1. Whether an organism grows under different conditions, 
2. The rate at which the organism grows, and 
3. The rate at which a metabolite is produced  
  
However, FBA assumes a steady-state and linear contraints, which does not take into account growth kinetics or kinetic rate laws in an actual cell. Dynamic flux balance analysis (dFBA) is an extension of FBA that resolves this issue, and takes into account the changes of metabolite concentrations and reaction fluxes as a function of time due to biochemical and regulatory changes in the cell.

Given a specific media and organism, we aim to create a dynamic database containing gap-filled models and the results of dFBA under various conditions, while:
* Providing an interactive graphical interface for visualizing gap-filled metabolic networks
* Facilitating the troubleshooting of why an organism grows/does not grow given specific environmental conditions
* Providing graphic visualizations of the simulated growth curves (biomass vs. time) and flux throughout a metabolic pathway  
![Workflow](Images/pipeline.png)  

This tutorial will be focusing on *Pseudomonas simiae WCS417* growing on a RCH2_defined_no_Carbon media (defined by Adam Deutschbauer in [Supplementary TableS18_Medias](https://www.nature.com/articles/s41586-018-0124-0#Sec36) ).  

# Setting up the Environment
You can create an Anaconda environment using:

    conda create --name [name]
    conda activate [name]
    
The following dependency installations assume you are using Anaconda. However, if you do not want to create an environment you can change each command to `pip install`

# Creating a Gap-filled Model
**As a USER, I would like to chose a model and media so that I can create a gap-filled model**   
In order to run dFBA, a gap-filled model is necessary to predict the uptake and secretion capabilities of an organism. Supplying a media file will guarantee that the model is able to reproduce growth in a experimentally verified media, i.e. gap-filling will identify the minimal set of biochemical reactions for a model to produce biomass in a specified media. 

[CarveMe](https://carveme.readthedocs.io/en/latest/usage.html) will be used to create such a model given a media. The user will supply the accession number or choose an organism from the database. However, the user can only choose from a set of pre-defined media (LB, M9, or RCH2) in the database. In the future, we would like to give the ability for the user to upload their own media file.
  
*CarveMe Dependencies*: diamond, cplex (requires license)

In [ ]:
pip install carveme

In [ ]:
conda install -c bioconda diamond

[Cplex](https://www.ibm.com/products/ilog-cplex-optimization-studio) can be downloaded from IBM and requires a license. Academic licenses are free.  
Once downloaded, you can install it using: 

    pip install -e [path to setup.py]

## Starting from Genome
Given a NCBI RefSeq accession number by the user, `CarveMe` will be used to first build a genome-scale metabolic model and then a gap-filled model in COBRA format.

In [ ]:
# Create metabolic model (not gap-filled) for storage in the database 
# if the same model needs to be run with a different media
!carve --refseq GCF_000698265.1 --cobra -o GCF_000698265.1.xml

In [ ]:
# Create a gap-filled model
!gapfill GCF_000698265.1.xml --cobra -m RCH2 --mediadb media_db.tsv 

This will create 4 files: 
1. GCF_000698265.1.faa.gz - protein FASTA file	
2. GCF_000698265.1.faa.tsv - gene matching by homology search using diamond
3. GCF_000698265.1.xml - metabolic model 
4. GCF_000698265.1_gapfill.xml - gap-filled metabolic model

We will store GCF_000698265.1.xml and GCF_000698265.1_gapfill.xml in our database.

### Retrieving Organism Name
While the RefSeq accession is a unique identifier, it is not easy to read. We will extract the metadata using `Biopython`. This will provide us the organism name for ease of selecting already-built models.

In [ ]:
conda install biopython 

In [ ]:
from Bio import Entrez

refseq = 'GCF_000698265.1'

handle = Entrez.esearch(db="assembly", term=refseq)
record = Entrez.read(handle)

# Get Assembly Summary
esummary_handle = Entrez.esummary(db="assembly", id=record['IdList'], report="full")
esummary_record = Entrez.read(esummary_handle)

# Organism and strain
organism = esummary_record['DocumentSummarySet']['DocumentSummary'][0]['SpeciesName']
try: 
    strain = esummary_record['DocumentSummarySet']['DocumentSummary'][0]['Biosource']['InfraspeciesList'][0]['Sub_value']
except:
    strain = ''
    
print(organism, strain)

From this, we can see that the RefSeq accession GCF_000698265.1 is *Pseudomonas simiae WCS417*. The species and strain will be stored in our database with the RefSeq accession as the unique identifier. The strain can be NULL, however the organism name cannot.

# Running dFBA
**As a USER, I would like to select my own set of conditions so that I can simulate an organism's growth under different conditions**  
Now that we have the gap-filled model, dFBA will be used to simulate growth and visualize growth curves with [CobraPy](https://opencobra.github.io/cobrapy/) on the backend. This will allow us to answer the following questions:
1. How does a microorganism grow under different sources/conditions?
2. What is the flow of a given metabolite through the metabolic network?
3. How do the growth rates of a specific organism compare across different conditions and/or media?
4. What are the reactions and their fluxes in a given pathway?
5. What are the upper and lower bounds of a particular reaction?  

In [ ]:
pip install cobra

## Growth Under Different Conditions


On the website, the user will be able to select one or more exchange reactions in the model and change the concentration via a drop down menu. Below is an example for providing 0.011 mmol/L of glucose and 0.03 mmol/L of sucrose.

In [2]:
import re
import cobra
import numpy as np
import pandas as pd

from math import exp

In [ ]:
# Load the gap-filled model
model = cobra.io.read_sbml_model('CarveMe/GCF_000698265.1_gapfill.xml')

In [3]:
# List of exchanges the user can choose from
for ex in model.exchanges:
    for metab in ex.metabolites:
        print(metab.id)

glc__D_e
h2o_e
14glucan_e
h_e
2pglyc_e
34dhbz_e
leu__L_e
3hoxpac_e
3mb_e
3oxoadp_e
4hbald_e
4hba_e
4hbz_e
ala__L_e
cl_e
arab__L_e
4abut_e
acac_e
acald_e
acgam_e
ac_e
chol_e
acolipa_e
acon_C_e
pi_e
actn__R_e
adn_e
ins_e
nh4_e
ade_e
ad_e
akg_e
mal__L_e
ala__D_e
gln__L_e
gly_e
ser__L_e
thr__L_e
arg__L_e
alltn_e
cgly_e
malt_e
malttr_e
arab__D_e
fe3_e
lys__L_e
orn_e
asp__L_e
phe__L_e
aso3_e
aso4_e
k_e
pro__L_e
ala_B_e
cellb_e
meoh_e
buts_e
butso3_e
but_e
bzal_e
bz_e
ca2_e
15dap_e
chols_e
cit_e
mg2_e
mn2_e
cobalt2_e
zn2_e
succ_e
for_e
lipa_cold_e
oxa_e
co2_e
colipa_e
csn_e
cu2_e
cu_e
cynt_e
o2_e
glu__L_e
diact_e
dad_2_e
dca_e
ddca_e
lac__D_e
drib_e
ser__D_e
enlipa_e
galur_e
hdca_e
hdcea_e
ocdca_e
ocdcea_e
ttdca_e
etha_e
eths_e
ethso3_e
etoh_e
h2_e
fe2_e
fe3dcit_e
enter_e
feenter_e
fru_e
fum_e
g3pc_e
gal_e
galctn__D_e
galctr__D_e
galct__D_e
gam_e
glcr_e
glcur_e
glyc3p_e
glyald_e
glyb_e
glyc2p_e
glyc__R_e
glyclt_e
glyc_e
gsn_e
gua_e
h2s_e
3hcinnm_e
pheme_e
hexs_e
his__L_e
4hphac_e
3hpppn_e
hxa

In [4]:
# Example list of exchanges and their concentrations defined by the user
exchanges = ['glc__D_e', 'sucr_e']
concentrations = [0.011, 0.03]

In [5]:
timeStep = 0.1

# Uptake reactions whose substrate concentrations do not change
excl = ['EX_co2','EX_o2','EX_h2o','EX_h']
exclUptakeRxns = []
for rxn in excl:
    regex = re.compile(rxn, flags=re.IGNORECASE)
    exclUptakeRxns.append(model.reactions.query(regex, attribute='id')[0])

In [6]:
user_exc = {}
# For each user defined value, find exchange rxns
for i in range(len(exchanges)):
    regex = re.compile(exchanges[i], flags=re.IGNORECASE)
    rxn = model.reactions.query(regex, attribute='id')[0]
    if rxn not in exclUptakeRxns:
        if concentrations[i] == 0 and -rxn.lower_bound > 0:
            concentrations[i] = 1
        user_exc[rxn] = {}
        user_exc[rxn]['concentration'] = concentrations[i]
        user_exc[rxn]['metabolite'] = model.metabolites.query(exchanges[i], attribute='id')[0]

In [7]:
# Initialize concentrations
excInd = list(set(model.exchanges) - set(exclUptakeRxns))
concentrationMatrix = np.zeros((int(48 / timeStep), len(excInd)))
for i in range(len(excInd)):
    rxn = excInd[i]
    try: 
        concentrationMatrix[0][i] = user_exc[rxn]['concentration']
    # Deal with reactions for which there are no initial concentrations
    except:
        if -rxn.lower_bound > 0:
            concentrationMatrix[0][i] = 1
             
biomassVec = [0.1]

In [8]:
# Initialize bounds - max is 1000
uptakeBound = np.zeros((int(48 / timeStep), len(excInd)), dtype=np.float64)
uptakeBound[0] = concentrationMatrix[0] / (0.1 * timeStep)
uptakeBound[0][uptakeBound[0] > 1000] = 1000

In [9]:
# Initialize flux table
uptakeFlux = np.zeros((int(48 / timeStep), len(excInd)), dtype=np.float64)

In [10]:
save = model.optimize()

# Run dFBA for 48 hours
for t in range(1, int(48 / timeStep)):
    # Get objective value
    sol = model.optimize()
    mu = sol.objective_value
    if mu == 0 or sol.status != 'optimal':
        sol = save
        break

    save = sol
    # Calculate biomass
    biomass = biomassVec[t-1] * exp(mu * timeStep)
    biomassVec.append(biomass) 

    # Update concentrations and save fluxes
    uptakeFlux[t-1] = np.array([model.reactions.get_by_id(exc.id).flux for exc in excInd]) 
    concentrationMatrix[t] = concentrationMatrix[t-1] - (uptakeFlux[t-1] / (mu * biomass * (1 - exp(mu * timeStep))))
    concentrationMatrix[t][concentrationMatrix[t] < 0] = 0 
    
    # Update bounds for uptake reactions
    uptakeBound[t] = concentrationMatrix[t]/(biomass * timeStep)
    uptakeBound[t][uptakeBound[t] > 1000] = 1000
    
    # Figure out if the computed bounds were above the original bounds
    for i in range(len(uptakeBound[t])):
        if (uptakeBound[t][i] > uptakeBound[t-1][i]) and (uptakeBound[t-1][i] > 0):
            uptakeBound[t][i] = uptakeBound[t-1][i]

    # < 1e-9 is considered 0
    uptakeBound[t][(abs(uptakeBound[t]) < 1e-9)] = 0

    # Update model
    for i in range(len(excInd)):
        model.exchanges.get_by_id(excInd[i].id).lower_bound = -uptakeBound[t][i]

In [11]:
# Convert arrays into a pandas dataframe
excRxnNames = [rxn.id for rxn in excInd]
excMetNames = [rxn.id[3:] for rxn in excInd]
concentration = pd.DataFrame(concentrationMatrix, columns =  excMetNames)
concentration

,colipa_e,indole_e,4abut_e,aso4_e,nh4_e,spmd_e,tre_e,cellb_e,glcr_e,pdima_e,...,maltpt_e,ad_e,ser__L_e,oaa_e,h_e,glyald_e,hdcea_e,man1p_e,4hphac_e,enlipa_e
0,1.0,1.0,1.000000,1.0,1.000000,1.0,1.000000,1.000000,1.0,1.0,...,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.0,1.0,1.000000,1.0
1,1.0,1.0,1.000000,1.0,1.000016,1.0,1.000000,1.000000,1.0,1.0,...,1.000000,1.000000,0.999994,1.0,0.999985,1.000000,1.0,1.0,1.000000,1.0
2,1.0,1.0,1.034743,1.0,0.965272,1.0,0.965257,0.965257,1.0,1.0,...,0.965257,0.965257,0.965252,1.0,0.965243,0.965257,1.0,1.0,0.965257,1.0
3,1.0,1.0,1.070789,1.0,0.929225,1.0,0.929210,0.929211,1.0,1.0,...,0.929211,0.929211,0.929205,1.0,0.929197,0.929211,1.0,1.0,0.929211,1.0
4,1.0,1.0,1.108288,1.0,0.891726,1.0,0.891711,0.891712,1.0,1.0,...,0.891712,0.891712,0.891707,1.0,0.891698,0.891712,1.0,1.0,0.891712,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
475,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0
476,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0
477,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0
478,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0


In [12]:
uptakeFlux = pd.DataFrame(uptakeFlux, columns = excRxnNames)
uptakeFlux

,EX_colipa_e,EX_indole_e,EX_4abut_e,EX_aso4_e,EX_nh4_e,EX_spmd_e,EX_tre_e,EX_cellb_e,EX_glcr_e,EX_pdima_e,...,EX_maltpt_e,EX_ad_e,EX_ser__L_e,EX_oaa_e,EX_h_e,EX_glyald_e,EX_hdcea_e,EX_man1p_e,EX_4hphac_e,EX_enlipa_e
0,0.0,0.0,0.000000,0.0,1000.000000,0.0,-12.480943,0.000000,0.0,0.0,...,0.000000,0.000000,-353.216238,0.0,-926.840328,0.000000,0.0,0.0,0.000000,0.0
1,0.0,0.0,0.035352,0.0,-0.035353,0.0,-0.035352,-0.035352,0.0,0.0,...,-0.035352,-0.035352,-0.035352,0.0,-0.035352,-0.035352,0.0,0.0,-0.035352,0.0
2,0.0,0.0,0.033492,0.0,-0.033492,0.0,-0.033492,-0.033492,0.0,0.0,...,-0.033492,-0.033492,-0.033492,0.0,-0.033491,-0.033492,0.0,0.0,-0.033492,0.0
3,0.0,0.0,0.031675,0.0,-0.031676,0.0,-0.031675,-0.031675,0.0,0.0,...,-0.031675,-0.031675,-0.031675,0.0,-0.031675,-0.031675,0.0,0.0,-0.031675,0.0
4,0.0,0.0,0.029892,0.0,-0.029892,0.0,-0.029892,-0.029892,0.0,0.0,...,-0.029892,-0.029892,-0.029891,0.0,-0.029891,-0.029892,0.0,0.0,-0.029892,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
475,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0
476,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0
477,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0
478,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0


In [13]:
uptake = pd.DataFrame(uptakeBound, columns =  excRxnNames)
uptake

,EX_colipa_e,EX_indole_e,EX_4abut_e,EX_aso4_e,EX_nh4_e,EX_spmd_e,EX_tre_e,EX_cellb_e,EX_glcr_e,EX_pdima_e,...,EX_maltpt_e,EX_ad_e,EX_ser__L_e,EX_oaa_e,EX_h_e,EX_glyald_e,EX_hdcea_e,EX_man1p_e,EX_4hphac_e,EX_enlipa_e
0,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,...,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
1,0.035352,0.035352,0.035352,0.035352,0.035353,0.035352,0.035352,0.035352,0.035352,0.035352,...,0.035352,0.035352,0.035352,0.035352,0.035352,0.035352,0.035352,0.035352,0.035352,0.035352
2,0.034697,0.034697,0.035352,0.034697,0.033492,0.034697,0.033492,0.033492,0.034697,0.034697,...,0.033492,0.033492,0.033492,0.034697,0.033491,0.033492,0.034697,0.034697,0.033492,0.034697
3,0.034088,0.034088,0.035352,0.034088,0.031676,0.034088,0.031675,0.031675,0.034088,0.034088,...,0.031675,0.031675,0.031675,0.034088,0.031675,0.031675,0.034088,0.034088,0.031675,0.034088
4,0.033522,0.033522,0.035352,0.033522,0.029892,0.033522,0.029892,0.029892,0.033522,0.033522,...,0.029892,0.029892,0.029891,0.033522,0.029891,0.029892,0.033522,0.033522,0.029892,0.033522
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
475,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
476,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
477,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
478,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


We will save these panda dataframes as csv files for storage in our database

In [14]:
concentration.T.to_csv('dFBA/concentrations.csv')
uptake.T.to_csv('dFBA/uptake.csv')
uptakeFlux.T.to_csv('dFBA/flux.csv')

# Graphical Outputs
dFBA concentration and biomass growth curves will be visualized using [plotly](https://plotly.com/python/). This will give a more interactive plot that can be embedded into the website. Furthermore, the fluxes from the dFBA results can be visualized in a metabolic pathway using [Escher](http://escher.github.io).

In [ ]:
pip install plotly

In [ ]:
pip install escher

In [14]:
import escher
import plotly.express as px

from escher import Builder
from plotly import graph_objects as go
from plotly.subplots import make_subplots

## Plot Concentrations and Biomass

In [16]:
# Plot concentrations as a function of time
fig = make_subplots(specs=[[{"secondary_y": True}]])

step = np.arange(len(biomassVec) + 1 * 0.1)
domain = [0]
for i in range(1, len(step) - 1):
    domain.append(domain[i-1] + 0.1)

# Plot the metabolites given by the user
for rxn in user_exc.keys():
    fig.add_trace(go.Scatter(x=domain, y=concentration[user_exc[rxn]['metabolite'].id],
                             mode="lines", name=user_exc[rxn]['metabolite'].name, line={"dash": "dot"}),
                  secondary_y=True)

# Plot biomass
fig.add_trace(go.Scatter(x=domain, y=biomassVec, mode="lines", name='Biomass'), secondary_y=False)

fig.update_xaxes(title_text='Time[h]')
fig.update_yaxes(title_text='Biomass [gL<sup>-1</sup>]', secondary_y=False)

fig.update_yaxes(title_text='Metabolites [mmolL<sup>-1</sup>]', secondary_y=True)

fig.write_html('dFBA/growth.html')
fig.show()

## Visualize Metabolic Pathways
Genome maps are a common caveat to visualizing metabolic pathways. Without a map of the reactions and metabolites in node format, an accurate genome representation cannot be given. To our knowledge, there is no map for *P. simiae*, therefore for this organism we will use *E. coli* for our example.

In [17]:
builder = Builder()
builder.map_name='e_coli_core.Core metabolism'

builder.model = model
builder.hide_secondary_metabolites = False
builder.hide_all_labels = False
builder.reaction_scale = [
    { 'type': 'min', 'color': '#000000', 'size': 12 },
    { 'type': 'median', 'color': '#ffffff', 'size': 20 },
    { 'type': 'max', 'color': '#ff0000', 'size': 25 }
]

builder.reaction_scale_preset = 'GaBuRd'
builder.reaction_scale = [
    {k: v * 3 if k == 'size' else v for k, v in x.items()}
    for x in builder.reaction_scale
]

In [18]:
# Get data to visualize
builder.reaction_data = sol.fluxes
builder.metabolite_data = concentration.T.loc[:, (concentration.T != 0).any(axis=0)].iloc[:,-1:]

In [19]:
builder.save_html('dFBA/example_map.html')

# Searching for Metabolites/Reactions
The user can also search for a specific metabolite or reaction in the model. We can output the results from the COBRA model, which contains the id, name, flux, etc. Furthermore, this query can be searched for in the BiGG database and external links for further information can be found.

## Specific Metabolite by ID

In [ ]:
model.metabolites.query('pyr_c', attribute='id')[0]

## Specific Metabolite by Name

In [ ]:
model.metabolites.query('Glucose', attribute='name')[0]

## Specific Reaction by ID

In [ ]:
model.reactions.query('ADCL', attribute='id')[0]